In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('model')

# Block 1: nodes with mixed scheduling
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)   # Processor Sharing
node3 = Queue(model, 'Queue2', SchedStrategy.DPS)  # Discriminatory Processor Sharing

# Block 2: classes with different populations
jobclass1 = ClosedClass(model, 'Class1', 2, node1, 0)  # 2 jobs
jobclass2 = ClosedClass(model, 'Class2', 1, node1, 0)  # 1 job

print("Network created with mixed scheduling:")
print(f"  {node1.get_name()}: Delay node")
print(f"  {node2.get_name()}: PS (Processor Sharing)")
print(f"  {node3.get_name()}: DPS (Discriminatory Processor Sharing)")
print(f"\nClass populations: Class1={jobclass1.getPopulation()}, Class2={jobclass2.getPopulation()}")

Network created with mixed scheduling:
  Delay: Delay node
  Queue1: PS (Processor Sharing)
  Queue2: DPS (Discriminatory Processor Sharing)

Class populations: Class1=2.0, Class2=1.0


In [3]:
# Set service distributions
node1.set_service(jobclass1, Exp.fitMean(1.0/3.0))   # Delay, Class1: mean=1/3
node1.set_service(jobclass2, Exp.fitMean(1.0/0.5))   # Delay, Class2: mean=1/0.5=2

# PS Queue - weights are ignored in PS scheduling
w1_ps = 5  # This weight is ignored since the node is PS
w2_ps = 1  # This weight is ignored since the node is PS
node2.set_service(jobclass1, Exp.fitMean(1.0/0.1), w1_ps)
node2.set_service(jobclass2, Exp.fitMean(1.0/1.0), w2_ps)

# DPS Queue - weights matter for service differentiation
w1_dps = 1  # Lower weight for Class1
w2_dps = 5  # Higher weight for Class2 (gets priority)
node3.set_service(jobclass1, Exp.fitMean(1.0/0.1), w1_dps)
node3.set_service(jobclass2, Exp.fitMean(1.0/1.0), w2_dps)

print("Service parameters configured:")
print(f"  Delay: Class1=Exp(3), Class2=Exp(0.5)")
print(f"  Queue1 (PS): Class1=Exp(0.1), Class2=Exp(1) - weights ignored")
print(f"  Queue2 (DPS): Class1=Exp(0.1,w={w1_dps}), Class2=Exp(1,w={w2_dps}) - Class2 has priority")

Service parameters configured:
  Delay: Class1=Exp(3), Class2=Exp(0.5)
  Queue1 (PS): Class1=Exp(0.1), Class2=Exp(1) - weights ignored
  Queue2 (DPS): Class1=Exp(0.1,w=1), Class2=Exp(1,w=5) - Class2 has priority


In [4]:
# Set up routing matrix
P = model.init_routing_matrix()

# Class1 routing: probabilistic from Delay (30% Queue1, 70% Queue2)
P.set(jobclass1, jobclass1, node1, node2, 0.3)  # Delay -> Queue1 (30%)
P.set(jobclass1, jobclass1, node1, node3, 0.7)  # Delay -> Queue2 (70%)
P.set(jobclass1, jobclass1, node2, node1, 1.0)  # Queue1 -> Delay (100%)
P.set(jobclass1, jobclass1, node3, node1, 1.0)  # Queue2 -> Delay (100%)

# Class2 routing: probabilistic from Delay (70% Queue1, 30% Queue2)
P.set(jobclass2, jobclass2, node1, node2, 0.7)  # Delay -> Queue1 (70%)
P.set(jobclass2, jobclass2, node1, node3, 0.3)  # Delay -> Queue2 (30%)
P.set(jobclass2, jobclass2, node2, node1, 1.0)  # Queue1 -> Delay (100%)
P.set(jobclass2, jobclass2, node3, node1, 1.0)  # Queue2 -> Delay (100%)

model.link(P)

print("Routing configured:")
print("  Class1: 30% Queue1, 70% Queue2 (more traffic to DPS queue)")
print("  Class2: 70% Queue1, 30% Queue2 (more traffic to PS queue)")
print("  All queues return 100% to Delay")

Routing configured:
  Class1: 30% Queue1, 70% Queue2 (more traffic to DPS queue)
  Class2: 70% Queue1, 30% Queue2 (more traffic to PS queue)
  All queues return 100% to Delay


In [5]:
# Configure multiple solvers for comparisonsolver_list = []# Add different solvers with their respective options# CTMCctmc_options = SolverCTMC.defaultOptions()ctmc_options.verbose = 1solver_list.append(SolverCTMC(model, ctmc_options))# JMTjmt_options = SolverJMT.defaultOptions()jmt_options.verbose = 1jmt_options.samples = 10000jmt_options.seed = 23000solver_list.append(SolverJMT(model, jmt_options))# Fluidfluid_options = SolverFluid.defaultOptions()fluid_options.verbose = 1solver_list.append(SolverFluid(model, fluid_options))# MVAmva_options = SolverMVA.defaultOptions()mva_options.verbose = 1solver_list.append(SolverMVA(model, mva_options))print(f"Configured {len(solver_list)} solvers for comparison")

=== Multi-Solver Analysis ===
Configured 4 solvers for comparison


In [6]:
# Solve with each solver and compare results
avg_tables = []

for s, solver in enumerate(solver_list):
    print(f'\n=== SOLVER: {solver.get_name()} ===')
    avg_table = solver.get_avg_table()
    avg_tables.append(avg_table)
    print(avg_table)


=== SOLVER: SolverCTMC ===
CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.042864s.
  Station JobClass    QLen    Util    RespT   ResidT    ArvR    Tput
0   Delay   Class1  0.0372  0.0372   0.3333   0.3333  0.1117  0.1117
1   Delay   Class2  0.5865  0.5865   2.0000   2.0000  0.2933  0.2933
2  Queue1   Class1  0.5346  0.3351  15.9547   4.7864  0.0335  0.0335
3  Queue1   Class2  0.2984  0.2053   1.4537   1.0176  0.2053  0.2053
4  Queue2   Class1  1.4282  0.7819  18.2661  12.7863  0.0782  0.0782
5  Queue2   Class2  0.1150  0.0880   1.3074   0.3922  0.0880  0.0880
  Station JobClass    QLen    Util    RespT   ResidT    ArvR    Tput
0   Delay   Class1  0.0372  0.0372   0.3333   0.3333  0.1117  0.1117
1   Delay   Class2  0.5865  0.5865   2.0000   2.0000  0.2933  0.2933
2  Queue1   Class1  0.5346  0.3351  15.9547   4.7864  0.0335  0.0335
3  Queue1   Class2  0.2984  0.2053   1.4537   1.0176  0.2053  0.2053
4  Queue2   Class1  1.4282  0.7819  18.2661  12.7863  0.0782  0